In [2]:
import pandas as pd
import os
from collections import Counter

def ensemble_target_values(csv_files):
    """
    여러 CSV 파일의 target 값을 majority voting 방식으로 앙상블합니다.
    
    Parameters:
    csv_files (list): CSV 파일 경로 리스트
    
    Returns:
    pandas.DataFrame: ID, text, target 값을 포함하는 DataFrame
    """
    # CSV 파일들 읽어오기
    dataframes = [pd.read_csv(file) for file in csv_files]
    
    # 모든 DataFrame의 공통 열 확인
    id_column = None
    text_column = None
    target_column = None
    
    for col in dataframes[0].columns:
        if all(col in df.columns for df in dataframes):
            if 'id' in col.lower():
                id_column = col
            elif 'text' in col.lower():
                text_column = col
            elif 'target' in col.lower():
                target_column = col
    

    # 각 ID별 target 값과 text 값 모으기
    ensemble_results = {}
    
    for df in dataframes:
        for _, row in df.iterrows():
            id_value = row[id_column]
            text_value = row[text_column]
            target_value = row[target_column]
            
            if id_value not in ensemble_results:
                ensemble_results[id_value] = {
                    'targets': [],
                    'texts': []
                }
            
            ensemble_results[id_value]['texts'].append(text_value)
            ensemble_results[id_value]['targets'].append(target_value)

    # Majority voting 수행
    final_results = {}
    
    for id_value, data in ensemble_results.items():
        # 가장 많은 target 값 찾기
        vote_counter = Counter(data['targets'])
        most_common = vote_counter.most_common(1)[0]
        
        # 가장 많은 값이 여러 개라면 첫 번째 파일의 값 선택
        if len(vote_counter) > 1 and list(vote_counter.values()).count(most_common[1]) > 1:
            final_target = dataframes[0].loc[
                dataframes[0][id_column] == id_value, target_column
            ].values[0]
        else:
            final_target = most_common[0]
        
        # text 값 처리 (첫 번째 파일의 text 사용)
        final_text = data['texts'][0] if data['texts'] else None
        
        final_results[id_value] = {
            target_column: final_target,
            text_column: final_text
        }
    
    # 결과 DataFrame 생성
    result_df = pd.DataFrame.from_dict(final_results, orient='index')
    result_df.index.name = id_column
    result_df.reset_index(inplace=True)
    
    return result_df

In [ ]:
csv_files = [
    'assemble_denoised_third_train.csv', 
    'assemble_denoised_sec_train.csv', 
    'assemble_denoised_fourth_train.csv'
]

# 앙상블 수행
ensemble_result = ensemble_target_values(csv_files)

# data augmentation
train_noisy_selection = pd.read_csv('output/train_noisy_selection.csv')
train_noisy_selection = train_noisy_selection.drop(columns=['Unnamed: 0'])
combined_df = pd.concat([ensemble_result, train_noisy_selection], ignore_index=True)
shuffled_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# 결과 저장
ensemble_result.to_csv('ensemble_targets.csv', index=False)

In [22]:
# 각 target별 개수와 비율 구하기
target_counts = shuffled_df['target'].value_counts()
target_ratios = shuffled_df['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
5         656  0.149738
1         649  0.148140
6         645  0.147227
4         636  0.145172
0         610  0.139238
2         607  0.138553
3         578  0.131933
